In [1039]:
import pandas as pd
import numpy as np 
import altair as alt
import warnings 
import seaborn as sns
from scipy import stats
import requests
import os
#import census_key
warnings.filterwarnings('ignore')
from urllib.request import urlopen
import zipfile
import requests
import xlrd
import openpyxl
from openpyxl import load_workbook
import requests, PyPDF2
from io import BytesIO

#pd.set_option('display.max_columns', 100)
pd.options.display.float_format = '{:.2f}'.format
pd.set_option("display.max_columns", 330)

In [1040]:
acs_2019 = pd.read_csv("raw_data/NE_social_explorer_acs.csv",dtype={"Geo_FIPS":str}) #from social explorer 

acs_2019.rename(columns={"Geo_FIPS":"fips_countysub",
                     "ACS19_5yr_B19013001":"plc_med_income",
                     "ACS19_5yr_B25064001":"plc_med_rent",
                     "ACS19_5yr_B25012011":"plc_renter_child_households",
                     "ACS19_5yr_B25115021":"plc_renter_child_single_parent_households_male",
                     "ACS19_5yr_B25115024":"plc_renter_child_single_parent_households_female",
                        "SE_B13004_001":"plc_povpop",
                           "SE_B13004_002":"plc_belowpov",
                           "ACS19_5yr_B01003001":"plc_pop",
                           "ACS19_5yr_B25003001":"plc_households",
                           "ACS19_5yr_B25003002":"plc_owners",
                           "ACS19_5yr_B25003003":"plc_renters",
                           "ACS19_5yr_B03002003":"plc_white",
                            "ACS19_5yr_B03002004":"plc_black",
                           "ACS19_5yr_B03002006":"plc_asian",
                            "ACS19_5yr_B03002005": "plc_amin",
                           "ACS19_5yr_B03002007":"plc_nhpil",
                           "ACS19_5yr_B03002009":"plc_other",
                            "ACS19_5yr_B03002012":"plc_hispanic"
                              },inplace=True)
#add male and female headed households together 
acs_2019["plc_renter_child_single_parent_households"] = acs_2019['plc_renter_child_single_parent_households_male'] + acs_2019["plc_renter_child_single_parent_households_female"]
acs_2019["plc_pov"] = acs_2019["plc_belowpov"]/acs_2019["plc_povpop"]

In [1041]:
acs_2019 = acs_2019[['fips_countysub',"Geo_NAME","Geo_STUSAB",'plc_pop',
'plc_households',
'plc_owners',
'plc_renters',
'plc_white',
'plc_black',
'plc_amin',
'plc_asian',
'plc_nhpil',
'plc_other',
'plc_hispanic',
'plc_povpop',
'plc_belowpov','plc_pov',
'plc_med_income',
'plc_renter_child_households',
'plc_med_rent',
'plc_renter_child_single_parent_households']]

In [1042]:
#drop out NE states and PR
acs_2019 = acs_2019[acs_2019.Geo_STUSAB.isin(["ct", "me", "ma", "nh", "ri", "vt"])]
#drop CDPs 
acs_2019 = acs_2019[~acs_2019.Geo_NAME.str.contains("CDP")]
#drop places that dont have a population 
acs_2019 = acs_2019[acs_2019.plc_pop > 0 ]
#standardize capitalization 
acs_2019.Geo_NAME = acs_2019.Geo_NAME.str.title()
#drop UT designation
acs_2019 = acs_2019[~acs_2019.Geo_NAME.str.contains("Ut")]
#looked for gores
acs_2019 = acs_2019[~acs_2019.Geo_NAME.str.contains("gore")]

In [1043]:
acs_2019.shape

(1540, 21)

For mass, all county subdivision are incorporated. For Maine County subdivisions with the UT designation are "unorganized territories" and should be considered unincorporated, but all others should be included.
For connecticut drop everything from Tolland and Windham counties, otherwise everythign else incorporated. 


In [1044]:
#load in county to subcounty mapping 
county = pd.read_csv("raw_data/full_NE_county_county_subdivision.csv")#from geocorr 
county["fips_countysub"] = county["county14"] + county["cousubfp14"]
county.rename(columns={"county":"county_FIPS",
                         "cntyname2":"cnty_name"},inplace=True)
county  = county[["fips_countysub","county_FIPS","cnty_name"]]
#additional_county_ma = pd.DataFrame(np.array([["2501724960", "25017", "Middlesex County"], ["2502156000", "25021", "Norfolk County"], ["2502308130", "25023", "Plymouth County"]]),
#                   columns=['fips_countysub', 'county_FIPS', "cnty_name"])
#county = pd.concat([county,additional_county_ma])

#manually add a few missing 

In [1045]:
#merge on county mapping 
acs_2019 = acs_2019.merge(county,on="fips_countysub",how="left")

In [1046]:
url = 'https://www.nh.gov/osi/resource-library/municipal/documents/unincorporated-places.pdf'#list of unicorporated palces in nh
response = requests.get(url)
my_raw_data = response.content

with BytesIO(my_raw_data) as data:
    read_pdf = PyPDF2.PdfFileReader(data)

    for page in range(read_pdf.getNumPages()):
        nh_unicorporated = read_pdf.getPage(page).extractText()
        
nh_unicorporated = nh_unicorporated.split("\n")
nh_unicorporated = [x.strip() for x in nh_unicorporated ]
nh_unicorporated = [x.replace("™","") for x in nh_unicorporated ]
fix_nh = ["Crawfords Purchase", "Cutts Grant" ,"Dixs Grant" ,'Dixville']
nh_unicorporated.extend(fix_nh)

In [1047]:
#remove townships from NH. #https://www.nh.gov/osi/resource-library/municipal/documents/unincorporated-places.pdf removed places from this list 
acs_2019 = acs_2019[~acs_2019.Geo_NAME.isin(nh_unicorporated)]

All in email, but two or three States are fully incorporated, but some are not. Remove Gores, “VT”, 

In NH, she thinks, “purchase, grant or locations, except hart location. 

Also townships, like cambridgeship 



### Load in CBDG data from 2019 and 2020

In [1048]:
#need to load in and clean cdbg 2019 
cdbg_place = pd.read_excel("raw_data/fy2019-formula-allocations-AllGrantees.xlsx", dtype={'KEY': str})
# fix DC KEY
cdbg_place['KEY']= cdbg_place.KEY.str.replace('119999','110006')
# convert CDBG20 and other HUD $$ columns to integers
hud_col = [x for x in cdbg_place.columns if x not in ['KEY','CNSRTKEY','NAME','STA']]
for col in hud_col:
    cdbg_place[col] = cdbg_place[col].replace({'\$':'',',':''}, regex=True)
    cdbg_place[col] = cdbg_place[col].fillna(0).astype(float)
# properly capitalize the name column
cdbg_place['NAME'] = cdbg_place['NAME'].str.title()

# rename columns for merge
cdbg_place.rename(columns={'NAME':'GEOG', 'STA':'stab'}, inplace= True)

# eliminate area descriptors  
cdbg_place['GEOG']= cdbg_place.GEOG.str.replace('Township','')
cdbg_place['GEOG']= cdbg_place.GEOG.str.replace('City','')
cdbg_place['GEOG']= cdbg_place.GEOG.str.replace('city','')
cdbg_place['GEOG']= cdbg_place.GEOG.str.replace('Village','')
cdbg_place['GEOG']= cdbg_place.GEOG.str.replace(" Town",'')
cdbg_place['GEOG']= cdbg_place.GEOG.str.replace(" town",'')
cdbg_place['GEOG']= cdbg_place.GEOG.str.replace(' Twp','')
cdbg_place['GEOG']= cdbg_place.GEOG.str.replace('Ft ','Fort ')
cdbg_place['GEOG']= cdbg_place.GEOG.str.replace('St ','St. ')
cdbg_place['GEOG']= cdbg_place.GEOG.str.replace('Htgs ','Heights')
cdbg_place['GEOG']= cdbg_place.GEOG.str.replace('Hts','Heights')
cdbg_place['GEOG']= cdbg_place.GEOG.str.replace(r"'","")
cdbg_place['GEOG']= cdbg_place.GEOG.str.replace(r" /County","")

# implement specific textual fixes
cdbg_place['GEOG']= cdbg_place.GEOG.str.replace('Mc Allen','McAllen')
cdbg_place['GEOG']= cdbg_place.GEOG.str.replace('Fond Du Lac','Fond du Lac')
cdbg_place['GEOG']= cdbg_place.GEOG.str.replace('Coeur DAlene','Coeur dAlene')
cdbg_place['GEOG']= cdbg_place.GEOG.str.replace('Jacksonville-Duval Count','Jacksonville')
cdbg_place['GEOG']= cdbg_place.GEOG.str.replace('Glen Falls','Glens Falls')
cdbg_place['GEOG']= cdbg_place.GEOG.str.replace('Dekalb','DeKalb')
cdbg_place['GEOG']= cdbg_place.GEOG.str.replace('Houma-Terrebonne','Houma')
cdbg_place['GEOG']= cdbg_place.GEOG.str.replace('Haverford','Haverford College')
cdbg_place['GEOG']= cdbg_place.GEOG.str.replace('Desoto','DeSoto')
cdbg_place['GEOG']= cdbg_place.GEOG.str.replace('Mckinney','McKinney')
cdbg_place['GEOG']= cdbg_place.GEOG.str.replace('Mckeesport','McKeesport')
cdbg_place['GEOG']= cdbg_place.GEOG.str.replace('Mckeesport','McKeesport')
cdbg_place['GEOG']= cdbg_place.GEOG.str.replace('Berwick Borough','Berwick')
cdbg_place['GEOG']= cdbg_place.GEOG.str.replace('District Of Columbia','Washington')
cdbg_place['GEOG']= cdbg_place.GEOG.str.replace("College College","College")

#strip spaces off end of GEOG
cdbg_place['GEOG'] = cdbg_place['GEOG'].str.strip()

cdbg_place = cdbg_place[cdbg_place.stab.isin(["CT", "ME", "MA", "NH", "RI", "VT"])]
cdbg_place.rename(columns={"GEOG":"NAME",
                          "stab":"STA"},inplace=True)
cdbg_place.drop(columns=["KEY","CNSRTKEY"],inplace=True)

In [1049]:
cdbg = pd.read_csv("raw_data/new_england_HUD.csv")
cdbg.drop(columns=["Unnamed: 0","KEY","CNSRTKEY","state","county","place"],inplace=True)
cdbg = cdbg.merge(cdbg_place,on=["NAME","STA"],how="left")
cdbg.rename(columns={"NAME":"clean_name",
                    'CDBG20':"plc_CDBG20", 
                        'HOME20':"plc_HOME20",
                        'ESG20':"plc_ESG20",
                        'CDBG_CV1':"plc_CDBG_CV1", 
                        'CDBG_CV2':"plc_CDBG_CV2", 
                        'CDBG_CV3':"plc_CDBG_CV3", 
                        'ESG_CV1':"plc_ESG_CV1", 
                        'ESG_CV2':"plc_ESG_CV2",
                        'CDBG19':"plc_CDBG19", 
                        'HOME19':"plc_HOME19", 
                        'ESG19':"plc_ESG19", 
                        'HOPWA19':"plc_HOPWA19", 
                        'HTF19':"plc_HTF19",
                     'CDBG19':"plc_CDBG19", 
                     'HOME19':"plc_HOME19", 
                     'ESG19':"plc_ESG19", 
                     'HOPWA19':"plc_HOPWA19", 
                     'HTF19':"plc_HTF19",
                    "STA":"Geo_STUSAB"},inplace=True)
cdbg["clean_name"] = cdbg.clean_name.str.replace(" Town","")
cdbg["clean_name"] = cdbg["clean_name"].str.replace(" City","")

In [1050]:
#remove town and city 

In [1051]:
#to merge cdbg funds on name and state need to match format 
acs_2019["clean_name"] = acs_2019.Geo_NAME.str.replace(" Town","")
acs_2019["clean_name"] = acs_2019["clean_name"].str.replace(" City","")
acs_2019.Geo_STUSAB = acs_2019.Geo_STUSAB.str.upper()

In [1052]:
acs_2019["place_state"] = acs_2019["clean_name"] +" "+ acs_2019["Geo_STUSAB"]
cdbg["place_state"] = cdbg["clean_name"] +" "+ cdbg["Geo_STUSAB"]

In [1053]:
place = acs_2019.merge(cdbg,on=["place_state"],how="left")

In [1054]:
def post_merge_col_name_fix(df):
    """
    Inputs:
        df (dataframe) - Dataframe post merge where the same named cols with _x and _y are identical in values

    This function looks at the typical naming conventions that occur during a merge with the same column names and fixes these inputs. 
    """
    for col in df.columns:
        df.rename(columns={col:col.replace("_x","")},inplace=True)
    
    df.drop(columns=[x for x in df.columns if "_y" in x],inplace=True)

    return df 

In [1055]:
place = post_merge_col_name_fix(place)

In [1056]:
### load in treasury ERAP data 
state_names = pd.read_csv("https://raw.githubusercontent.com/jasonong/List-of-US-States/master/states.csv")

erap1 = pd.read_excel("raw_data/November-ERA-Data.xlsx",sheet_name="ERA1 State & Local Reporting")
erap2 = pd.read_excel("raw_data/November-ERA-Data.xlsx",sheet_name="ERA2 State & Local Reporting")

def make_header(df):
    """
    do some basic cleanup of the header row 
    """
    new_header = df.iloc[0] #grab the first row for the header
    df = df[1:] #take the data less the header row
    df.columns = new_header
    return df
erap1 = make_header(erap1)
erap2 = make_header(erap2)
erap1["State"] = ""
erap2["State"] = ""

excel_file_erap1 = 'raw_data/November-ERA-Data.xlsx' 
wb_erap1 = load_workbook(excel_file_erap1, data_only = True)
sh_erap1 = wb_erap1['ERA1 State & Local Reporting']

#need to add labels for states - here im looking at color changes as in 
#excel sheet each state is highlighted, so we can change label we want whenever, we index
#through and see a new color 
state_name = "California"
for i in range(erap1.shape[0]):
    if i == 0:
        continue
    color_in_hex =sh_erap1['A'+str(i)].fill.start_color.tint
    if color_in_hex > 0:
        state_name = sh_erap1['A'+str(i)].value
    erap1.loc[i, "State"] = state_name    
    
excel_file_erap2 = 'raw_data/November-ERA-Data.xlsx' 
wb_erap2 = load_workbook(excel_file_erap2, data_only = True)
sh_erap2 = wb_erap1['ERA2 State & Local Reporting']

state_name = "California"
for i in range(erap2.shape[0]):
    if i == 0:
        continue
    color_in_hex =sh_erap2['A'+str(i)].fill.start_color.tint
    if color_in_hex > 0:
        state_name = sh_erap2['A'+str(i)].value
    erap2.loc[i,"State"] = state_name   
erap1 = make_header(erap1)
erap2 = make_header(erap2)
erap1.rename(columns={"California":"State"},inplace=True)
erap2.rename(columns={"California":"State"},inplace=True)
erap1 = erap1[erap1.Location != "Grand Total"]
erap2 = erap2[erap2.Location != "Grand Total"]
erap1=erap1.dropna(axis=1, how='all')
erap2=erap2.dropna(axis=1, how='all')
#merge data and drop DC 
erap1 = erap1.merge(state_names,on="State")
erap2 = erap2.merge(state_names,on="State")
erap1 = erap1[erap1.Location != "State/DC"]
erap2 = erap2[erap2.Location != "State/DC"]

#sum up funds and number of houesholds funds disbursed to 
erap1["Total Number of Households ERAP1 (Direct Allocation)"] = erap1[[x for x in erap1 if "Total" in x]].sum(axis=1)
erap2["Total Number of Households ERAP2 (Direct Allocation)"] = erap2[[x for x in erap2 if "Total" in x]].sum(axis=1)
erap1["ERAP1_total_funds (Direct Allocation)"] = erap1[[x for x in erap1.columns if "Amount" in x]].sum(axis=1)
erap2["ERAP2_total_funds (Direct Allocation)"] = erap2[[x for x in erap2.columns if "Amount" in x]].sum(axis=1)
#flag footnote six and then strip out 
#6Local government recipient that has redirected or will redirect ERA 1 funds to their county or state government. only exits in erap1
erap1["ERAP1_redirected_funds"] = np.where(erap1.Location.str.contains("6"),1,0)
erap1.Location = erap1.Location.str.strip("6")

erap1.rename(columns={"Assistance to Households Expenditure Ratio as of November 305,7":"Assistance to Households Expenditure Ratio as of November ERAP_1 (Direct Allocation)"},inplace=True)
#need to calculate this for ERAP 2 because no longer provided- calcuate figure
#for ERAP2 using footnote 5 for ERAP 1 "5Calculated as the sum of Q1, April, May, June, July, August, September, October, and November expenditure for assistance to households divided by 90% of the ERA1 allocation amount."
erap2["Assistance to Households Expenditure Ratio as of November ERAP_2 (Direct Allocation)"] = erap2["ERAP2_total_funds (Direct Allocation)"]/ (erap2["Full ERA2 Allocation, Award, and Disbursement \n$ Millions4"] * 0.9)

total = erap1.merge(erap2,on=["Location","State","Abbreviation"],how="outer")

#keep only cols i want to merge onto the other 990 data
total_subset = total[["Location","ERAP1_total_funds (Direct Allocation)",'Assistance to Households Expenditure Ratio as of November ERAP_1 (Direct Allocation)',
      'Total Number of Households ERAP1 (Direct Allocation)','Assistance to Households Expenditure Ratio as of November ERAP_2 (Direct Allocation)',
      'ERAP2_total_funds (Direct Allocation)','Total Number of Households ERAP2 (Direct Allocation)',"ERAP1_redirected_funds","State","Abbreviation"]]

total_subset_merge = total_subset.rename(columns={"Location":"County"})

total_subset_merge["County Name"] = total_subset_merge["County"] +" "+ total_subset_merge["Abbreviation"]
total_subset_merge = total_subset_merge.drop(columns={"County","State","Abbreviation"})
#fix mis name 
total_subset_merge.replace({"County Name":{
                                         'Dekalb County GA':"DeKalb County GA"}},inplace=True)

#clean up and fix type of expenditure ratio 

total_subset_merge['Assistance to Households Expenditure Ratio as of November ERAP_1 (Direct Allocation)'] = total_subset_merge['Assistance to Households Expenditure Ratio as of November ERAP_1 (Direct Allocation)'].astype(str).str.strip("%")
total_subset_merge['Assistance to Households Expenditure Ratio as of November ERAP_2 (Direct Allocation)'] = total_subset_merge['Assistance to Households Expenditure Ratio as of November ERAP_2 (Direct Allocation)'].astype(str).str.strip("%")
total_subset_merge['Assistance to Households Expenditure Ratio as of November ERAP_1 (Direct Allocation)'] = total_subset_merge['Assistance to Households Expenditure Ratio as of November ERAP_1 (Direct Allocation)'].astype(str).str.strip("+")
total_subset_merge['Assistance to Households Expenditure Ratio as of November ERAP_2 (Direct Allocation)'] = total_subset_merge['Assistance to Households Expenditure Ratio as of November ERAP_2 (Direct Allocation)'].astype(str).str.strip("+")
total_subset_merge['Assistance to Households Expenditure Ratio as of November ERAP_1 (Direct Allocation)'] = total_subset_merge['Assistance to Households Expenditure Ratio as of November ERAP_1 (Direct Allocation)'].astype(float)
total_subset_merge['Assistance to Households Expenditure Ratio as of November ERAP_2 (Direct Allocation)'] = total_subset_merge['Assistance to Households Expenditure Ratio as of November ERAP_2 (Direct Allocation)'].astype(float)

In [1057]:
erap_new = total_subset_merge[total_subset_merge["County Name"].str.contains("City")]


In [1058]:
#Boston is the only treasury data that maps to this NE data. 
erap_new = erap_new[erap_new["County Name"].str.contains("MA")]

In [1059]:
erap_new.rename(columns={"County Name":"Geo_NAME"},inplace=True)
erap_new.replace({"Geo_NAME":{"City of Boston MA":"Boston City"}},inplace=True)

In [1060]:
place = place.merge(erap_new,on="Geo_NAME",how="left")
place.rename(columns={'ERAP1_total_funds (Direct Allocation)':"plc_erap1_funds",
       'Assistance to Households Expenditure Ratio as of November ERAP_1 (Direct Allocation)':"plc_ratio_erap_1",
       'Total Number of Households ERAP1 (Direct Allocation)':"plc_hh_erap_1",
       'Assistance to Households Expenditure Ratio as of November ERAP_2 (Direct Allocation)':"plc_ratio_erap_2",
       'ERAP2_total_funds (Direct Allocation)':"plc_erap2_funds", 'Total Number of Households ERAP2 (Direct Allocation)':"plc_hh_erap_2"},inplace=True)

### Add AMI data place 

In [1061]:
#read in AMI data, create county level FIPS 
ami = pd.read_excel("raw_data/hud_ami_2019.xlsx",dtype={"fips2010":str})
#ami.State = ami.State.astype(str).str.zfill(2)
#ami.County = ami.County.astype(str).str.zfill(3)
#ami["FIPS"] = ami.State + ami.County
ami = ami[["Metro_Area_Name","l50_4","fips2010"]]
#ami = ami.drop_duplicates(subset=["county14"])
ami.rename(columns={"fips2010":"fips_countysub"},inplace=True)

In [1062]:
#additional_county_ma = pd.DataFrame(np.array([["2501724960", "25017", "Middlesex County"], ["2502156000", "25021", "Norfolk County"], ["2502308130", "25023", "Plymouth County"]]),
#                   columns=['fips_countysub', 'county_FIPS', "cnty_name"])
#county = pd.concat([county,additional_county_ma]) 

In [1063]:
additional_ami = pd.DataFrame(np.array([["Boston-Cambridge-Quincy, MA-NH HUD Metro FMR Area",59250,"2501724960"],
                                        ["Boston-Cambridge-Quincy, MA-NH HUD Metro FMR Area",59250,"2502156000"],
                                        ["Brockton, MA HUD Metro FMR Area",46250,"2502308130"]]),
                                       columns=['Metro_Area_Name', 'l50_4', "fips_countysub"])
ami = pd.concat([ami,additional_ami])

In [1064]:
ami.l50_4 = ami.l50_4.astype(int)

In [1065]:
place = place.merge(ami,on="fips_countysub",how="left")

In [1066]:
se_data = pd.read_csv("raw_data/NE_renter_income_levels.csv",dtype={"Geo_FIPS":str})
se_data= se_data[["Geo_FIPS","Geo_NAME","Geo_STUSAB",
                'SE_A14001_001',
 'SE_A14001_002',
 'SE_A14001_003',
 'SE_A14001_004',
 'SE_A14001_005',
 'SE_A14001_006',
 'SE_A14001_007',
 'SE_A14001_008',
 'SE_A14001_009',
 'SE_A14001_010',
 'SE_A14001_011',
 'SE_A14001_012',
 'SE_A14001_013',
 'SE_A14001_014',
 'SE_A14001_015',
 'SE_A14001_016',
 'SE_A14001_017',
 'ACS19_5yr_B25118001',
 'ACS19_5yr_B25118002',
 'ACS19_5yr_B25118003',
 'ACS19_5yr_B25118004',
 'ACS19_5yr_B25118005',
 'ACS19_5yr_B25118006',
 'ACS19_5yr_B25118007',
 'ACS19_5yr_B25118008',
 'ACS19_5yr_B25118009',
 'ACS19_5yr_B25118010',
 'ACS19_5yr_B25118011',
 'ACS19_5yr_B25118012',
 'ACS19_5yr_B25118013',
 'ACS19_5yr_B25118014',
 'ACS19_5yr_B25118015',
 'ACS19_5yr_B25118016',
 'ACS19_5yr_B25118017',
 'ACS19_5yr_B25118018',
 'ACS19_5yr_B25118019',
 'ACS19_5yr_B25118020',
 'ACS19_5yr_B25118021',
 'ACS19_5yr_B25118022',
 'ACS19_5yr_B25118023',
 'ACS19_5yr_B25118024',
 'ACS19_5yr_B25118025']]

se_data.rename(columns={'SE_A14001_001':"Total_households_2019",
                       'SE_A14001_002':"Total_income_under_10k",
                       'SE_A14001_003':"Total_income_10k_15k", 
                       'SE_A14001_004':"Total_income_15k_20k", 
                       'SE_A14001_005':"Total_income_20k_25k", 
                       'SE_A14001_006':"Total_income_25k_30k",
                       'SE_A14001_007':"Total_income_30k_35k", 
                       'SE_A14001_008':"Total_income_35k_40k", 
                       'SE_A14001_009':"Total_income_40k_45k", 
                       'SE_A14001_010':"Total_income_45k_50k",
                       'SE_A14001_011':"Total_income_50k_60k", 
                       'SE_A14001_012':"Total_income_60k_75k", 
                       'SE_A14001_013':"Total_income_75k-100k", 
                       'SE_A14001_014':"Total_income_100k_125k",
                       'SE_A14001_015':"Total_income_125k_150k", 
                       'SE_A14001_016':"Total_income_150k_200k", 
                       'SE_A14001_017':"Total_income_200k+",
                       'ACS19_5yr_B25118014':"Total_renter_occupied_Households_2019", 
                       'ACS19_5yr_B25118015':"Renter_total_income_under_5k", 
                       'ACS19_5yr_B25118016':"Renter_total_income_5k_10k",
                       'ACS19_5yr_B25118017':"Renter_total_income_10k_15k", 
                       'ACS19_5yr_B25118018':"Renter_total_income_15k_20k", 
                       'ACS19_5yr_B25118019':"Renter_total_income_20k_25k",
                       'ACS19_5yr_B25118020':"Renter_total_income_25k_35k", 
                       'ACS19_5yr_B25118021':"Renter_total_income_35k_50k", 
                       'ACS19_5yr_B25118022':"Renter_total_income_50k_75k",
                       'ACS19_5yr_B25118023':"Renter_total_income_75k_100k", 
                       'ACS19_5yr_B25118024':"Renter_total_income_100k_150k", 
                       'ACS19_5yr_B25118025':"Renter_total_income_150k+"},inplace=True)
se_data.rename(columns={"Geo_FIPS":"fips_countysub"},inplace=True)
#se_data = se_data.merge(ami,on="fips_place",how="left")
#places_final = se_data.drop_duplicates(subset=["fips_place","Total_income_under_10k"])

In [1067]:
place_income_merge = place[["fips_countysub","l50_4"]]

In [1068]:
se_data  = place_income_merge.merge(se_data,on="fips_countysub",how="left")#added left 

In [1069]:
places_final_total = se_data[["fips_countysub",'l50_4', 'Total_income_under_10k',
       'Total_income_10k_15k', 'Total_income_15k_20k', 'Total_income_20k_25k',
       'Total_income_25k_30k', 'Total_income_30k_35k', 'Total_income_35k_40k',
       'Total_income_40k_45k', 'Total_income_45k_50k', 'Total_income_50k_60k',
       'Total_income_60k_75k', 'Total_income_75k-100k',
       'Total_income_100k_125k', 'Total_income_125k_150k',
       'Total_income_150k_200k', 'Total_income_200k+']]

places_final_rental = se_data[["fips_countysub",'l50_4','Renter_total_income_under_5k',
       'Renter_total_income_5k_10k', 'Renter_total_income_10k_15k',
       'Renter_total_income_15k_20k', 'Renter_total_income_20k_25k',
       'Renter_total_income_25k_35k', 'Renter_total_income_35k_50k',
       'Renter_total_income_50k_75k', 'Renter_total_income_75k_100k',
       'Renter_total_income_100k_150k', 'Renter_total_income_150k+']]

In [1070]:
places_final_rental_melt=places_final_rental.melt(id_vars=["l50_4","fips_countysub"])
#second number 9,999
lower_dict_renter={'Renter_total_income_under_5k':0, 
'Renter_total_income_5k_10k':5000,
'Renter_total_income_10k_15k':10000, 
'Renter_total_income_15k_20k':15000,
'Renter_total_income_20k_25k':20000, 
'Renter_total_income_25k_35k':25000,
'Renter_total_income_35k_50k':35000, 
'Renter_total_income_50k_75k':50000,
'Renter_total_income_75k_100k':75000, 
'Renter_total_income_100k_150k':100000,
'Renter_total_income_150k+':150000}

upper_dict_renter={'Renter_total_income_under_5k':4999, 
'Renter_total_income_5k_10k':9999,
'Renter_total_income_10k_15k':14999, 
'Renter_total_income_15k_20k':19999,
'Renter_total_income_20k_25k':24999, 
'Renter_total_income_25k_35k':34999,
'Renter_total_income_35k_50k':49999, 
'Renter_total_income_50k_75k':74999,
'Renter_total_income_75k_100k':99999, 
'Renter_total_income_100k_150k':149990,
'Renter_total_income_150k+':200000}

def calc_bounds(l50_4,value,lower_bound,upper_bound):
    if l50_4 > upper_bound:
        return value
    if l50_4 <= upper_bound:
        if l50_4 > lower_bound:
            tmp = upper_bound - lower_bound
            tmp2 = np.abs(lower_bound - l50_4)
            return round((tmp2/tmp)*value)
        else:
            return 0
        
places_final_rental_melt["lower_bound"] =  places_final_rental_melt['variable'].map(lower_dict_renter)
places_final_rental_melt["upper_bound"] =  places_final_rental_melt['variable'].map(upper_dict_renter)
#calculate lower and upper bound 
places_final_rental_melt['Renters_Below_50%ami'] = places_final_rental_melt.apply(lambda x: calc_bounds(x.l50_4,x.value,x.lower_bound,x.upper_bound), axis=1)
#get total sum of rentersd below 50% AMI 
renter_counts = places_final_rental_melt[["fips_countysub","Renters_Below_50%ami"]].groupby("fips_countysub").sum().reset_index()
#create mapping of geocode to total number of renters 
places_final_total_renter = place[["plc_renters","fips_countysub"]]
places_final_total_renter_grouped = places_final_total_renter.merge(renter_counts,on="fips_countysub",how="left")#added left 
places_final_total_renter_grouped["Percent_of_Renter_Households_Below_50%AMI"] = 100*(places_final_total_renter_grouped["Renters_Below_50%ami"]/places_final_total_renter_grouped["plc_renters"])

### now same thing for total number of households 

In [1071]:
places_final_total_melt = places_final_total.melt(id_vars=['fips_countysub', 'l50_4'])
lower_dict_total = {'Total_income_under_10k':0, 
                    'Total_income_10k_15k':10000,
                    'Total_income_15k_20k':15000, 
                    'Total_income_20k_25k':20000, 
                    'Total_income_25k_30k':25000,
                    'Total_income_30k_35k':30000, 
                    'Total_income_35k_40k':35000, 
                    'Total_income_40k_45k':40000,
                    'Total_income_45k_50k':45000, 
                    'Total_income_50k_60k':50000, 
                    'Total_income_60k_75k':60000,
                    'Total_income_75k-100k':75000, 
                    'Total_income_100k_125k':100000,
                    'Total_income_125k_150k':125000, 
                    'Total_income_150k_200k':150000,
                    'Total_income_200k+':200000
    
}
upper_dict_total = {'Total_income_under_10k':9999, 
                    'Total_income_10k_15k':14999,
                    'Total_income_15k_20k':19999, 
                    'Total_income_20k_25k':24999, 
                    'Total_income_25k_30k':29999,
                    'Total_income_30k_35k':34999, 
                    'Total_income_35k_40k':39999, 
                    'Total_income_40k_45k':44999,
                    'Total_income_45k_50k':49999, 
                    'Total_income_50k_60k':59999, 
                    'Total_income_60k_75k':74999,
                    'Total_income_75k-100k':99999, 
                    'Total_income_100k_125k':124999,
                    'Total_income_125k_150k':149999, 
                    'Total_income_150k_200k':199999,
                    'Total_income_200k+':300000
    
}
places_final_total_melt["lower_bound"]= places_final_total_melt['variable'].map(lower_dict_total)
places_final_total_melt["upper_bound"]= places_final_total_melt['variable'].map(upper_dict_total)
#calculate lower and upper bound 
places_final_total_melt['Total_Households_Below_50%ami'] = places_final_total_melt.apply(lambda x: calc_bounds(x.l50_4,x.value,x.lower_bound,x.upper_bound), axis=1)
#get total sum of rentersd below 50% AMI 
total_counts = places_final_total_melt[["fips_countysub","Total_Households_Below_50%ami"]].groupby("fips_countysub").sum().reset_index()
#create mapping of geocode to total number of renters 
places_final_total = place[["plc_households","fips_countysub"]]
places_final_total_grouped = total_counts.merge(places_final_total,on="fips_countysub")
places_final_total_grouped["Percent_of_Households_Below_50%AMI"] = 100*(places_final_total_grouped["Total_Households_Below_50%ami"]/places_final_total_grouped["plc_households"])

In [1072]:
final_df = places_final_total_renter_grouped.merge(places_final_total_grouped,on="fips_countysub",how="left")#added left 

In [1073]:
final_df = final_df[["fips_countysub","Total_Households_Below_50%ami","Renters_Below_50%ami"]]

In [1074]:
final_df.rename(columns={"Total_Households_Below_50%ami":"plc_sub50_ami",
                    "Renters_Below_50%ami":"plc_sub50_ami_renter"},inplace=True)

In [1075]:
place = place.merge(final_df,on=["fips_countysub"],how="left")

In [1076]:
place.rename(columns={"Metro_Area_Name":"metro_area_name",
                     "l50_4":"50%AMI_Threshold",
                     "Geo_NAME":"plc_name",
                     "cnty_name":"cntyname",
                     "county_FIPS":"county_fips",
                     "Geo_STUSAB":"stab"},inplace=True)

In [1077]:
len(['stab','metro_area_name', 'cntyname', 'county_fips', 'plc_name', 'fips_countysub', 'plc_pop',
       'plc_households', 'plc_owners', 'plc_renters', 'plc_white', 'plc_black',
       'plc_amin', 'plc_asian', 'plc_nhpil', 'plc_hispanic', 'plc_other',
       'plc_povpop', 'plc_belowpov', 'plc_pov', 'plc_med_income',
       'plc_med_rent', 'plc_renter_child_households',
       'plc_renter_child_single_parent_households','plc_sub50_ami', 'plc_sub50_ami_renter',"50%AMI_Threshold",'plc_CDBG20', 'plc_HOME20',
       'plc_ESG20', 'plc_CDBG_CV1', 'plc_CDBG_CV2', 'plc_CDBG_CV3',
       'plc_ESG_CV1', 'plc_ESG_CV2', 'plc_CDBG19', 'plc_HOME19', 'plc_ESG19',
       'plc_HOPWA19', 'plc_HTF19', 'plc_erap1_funds', 'plc_ratio_erap_1',
       'plc_hh_erap_1', 'plc_erap2_funds', 'plc_ratio_erap_2', 'plc_hh_erap_2',
       'Entitlement_Community_Flag'])

47

In [1078]:
#place.to_csv("processed_data/NE_Places_final_1_26_22.csv")

In [1079]:
#additional_county_ma = pd.DataFrame(np.array([["2501724960", "25017", "Middlesex County"], ["2502156000", "25021", "Norfolk County"], ["2502308130", "25023", "Plymouth County"]]),
#                   columns=['fips_countysub', 'county_FIPS', "cnty_name"])
#county = pd.concat([county,additional_county_ma])

In [1080]:
place.at[854,"cntyname"]= "Middlesex County"
place.at[854,"county_fips"]= "25017"
place.at[912,"cntyname"]= "Norfolk County"
place.at[912,"county_fips"]= "25021"
place.at[921,"cntyname"]= "Plymouth County"
place.at[921,"county_fips"]= "25023"

In [1081]:
place = place[['stab','metro_area_name', 'cntyname', 'county_fips', 'plc_name', 'fips_countysub', 'plc_pop',
       'plc_households', 'plc_owners', 'plc_renters', 'plc_white', 'plc_black',
       'plc_amin', 'plc_asian', 'plc_nhpil', 'plc_hispanic', 'plc_other',
       'plc_povpop', 'plc_belowpov', 'plc_pov', 'plc_med_income',
       'plc_med_rent', 'plc_renter_child_households',
       'plc_renter_child_single_parent_households','plc_sub50_ami', 'plc_sub50_ami_renter',"50%AMI_Threshold",'plc_CDBG20', 'plc_HOME20',
       'plc_ESG20', 'plc_CDBG_CV1', 'plc_CDBG_CV2', 'plc_CDBG_CV3',
       'plc_ESG_CV1', 'plc_ESG_CV2', 'plc_CDBG19', 'plc_HOME19', 'plc_ESG19',
       'plc_HOPWA19', 'plc_HTF19', 'plc_erap1_funds', 'plc_ratio_erap_1',
       'plc_hh_erap_1', 'plc_erap2_funds', 'plc_ratio_erap_2', 'plc_hh_erap_2']]

In [1082]:
#place.to_csv("processed_data/NE_Places_final_1_27_22.csv")

In [1083]:
place.shape

(1537, 46)

In [1084]:
#50%AMI_Threshold                                3
place.mean()

county_fips                                         inf
fips_countysub                                      inf
plc_pop                                         9618.28
plc_households                                  3739.41
plc_owners                                      2441.11
plc_renters                                     1298.30
plc_white                                       7214.45
plc_black                                        605.90
plc_amin                                          19.67
plc_asian                                        459.64
plc_nhpil                                          3.01
plc_hispanic                                    1061.29
plc_other                                        202.76
plc_povpop                                      9302.47
plc_belowpov                                     954.40
plc_pov                                            0.09
plc_med_income                                 73206.93
plc_med_rent                                    